# Proyecto de Python learning

## Librerias

In [95]:
import kagglehub
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SequentialFeatureSelector



## Importar data

In [96]:
path = kagglehub.dataset_download("emonsharkar/python-learning-and-exam-performance-dataset")
df = pd.read_csv(os.path.join(path, os.listdir(path)[0]))
df.head()


,student_id,age,country,prior_programming_experience,weeks_in_course,hours_spent_learning_per_week,practice_problems_solved,projects_completed,tutorial_videos_watched,uses_kaggle,participates_in_discussion_forums,debugging_sessions_per_week,self_reported_confidence_python,final_exam_score,passed_exam
0,1,54,Pakistan,Beginner,12,12.2,60,1,24,0,1,4,10,53.8,0
1,2,44,Nigeria,NaN,15,14.2,52,1,32,0,0,5,7,31.9,0
2,3,30,India,Intermediate,5,7.7,71,1,43,0,0,6,3,59.4,0
3,4,23,Nigeria,Beginner,4,5.6,64,2,40,1,0,3,10,58.8,0
4,5,36,UK,NaN,3,3.5,62,1,47,1,0,3,7,24.8,0


In [97]:
df.shape

(3000, 15)

In [98]:
df.isnull().sum()

student_id                              0
age                                     0
country                                 0
prior_programming_experience         1062
weeks_in_course                         0
hours_spent_learning_per_week           0
practice_problems_solved                0
projects_completed                      0
tutorial_videos_watched                 0
uses_kaggle                             0
participates_in_discussion_forums       0
debugging_sessions_per_week             0
self_reported_confidence_python         0
final_exam_score                        0
passed_exam                             0
dtype: int64

In [99]:
df.describe(include = "all").round(0).T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
student_id,3000.0,NaN,NaN,NaN,1500.0,866.0,1.0,751.0,1500.0,2250.0,3000.0
age,3000.0,NaN,NaN,NaN,35.0,11.0,16.0,26.0,36.0,45.0,54.0
country,3000,10,Brazil,339,NaN,NaN,NaN,NaN,NaN,NaN,NaN
prior_programming_experience,1938,3,Beginner,1034,NaN,NaN,NaN,NaN,NaN,NaN,NaN
weeks_in_course,3000.0,NaN,NaN,NaN,8.0,4.0,1.0,5.0,8.0,12.0,15.0
hours_spent_learning_per_week,3000.0,NaN,NaN,NaN,7.0,3.0,0.0,5.0,7.0,9.0,17.0
practice_problems_solved,3000.0,NaN,NaN,NaN,60.0,8.0,35.0,55.0,60.0,65.0,87.0
projects_completed,3000.0,NaN,NaN,NaN,2.0,1.0,0.0,1.0,2.0,3.0,9.0
tutorial_videos_watched,3000.0,NaN,NaN,NaN,40.0,6.0,19.0,35.0,40.0,44.0,63.0
uses_kaggle,3000.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,1.0


In [100]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   student_id                         3000 non-null   int64  
 1   age                                3000 non-null   int64  
 2   country                            3000 non-null   object 
 3   prior_programming_experience       1938 non-null   object 
 4   weeks_in_course                    3000 non-null   int64  
 5   hours_spent_learning_per_week      3000 non-null   float64
 6   practice_problems_solved           3000 non-null   int64  
 7   projects_completed                 3000 non-null   int64  
 8   tutorial_videos_watched            3000 non-null   int64  
 9   uses_kaggle                        3000 non-null   int64  
 10  participates_in_discussion_forums  3000 non-null   int64  
 11  debugging_sessions_per_week        3000 non-null   int64

## Pipeline

### Creacion de Pipes

#### Clasico

In [101]:
drop_cols = ["prior_programming_experience", "student_id"]


num_var = make_column_selector(dtype_include="number")
cat_var = make_column_selector(dtype_include="object")

pipe_drop = FunctionTransformer(
                lambda x: x.drop(drop_cols, axis = 1, errors = "ignore")
                                )

pipe_cat = Pipeline(steps = [
                    ("cat_imputer", SimpleImputer(strategy= "most_frequent")),
                    ("ohe", OneHotEncoder()),
                    ]
                    )

pip_num = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median"))])

pp = ColumnTransformer( transformers= [
    #("drop", "drop", drop_cols),
    ("categ", pipe_cat, cat_var),
    ("num", pip_num, num_var)],
    remainder= "passthrough"
)

pipe_final = Pipeline(steps = [

                ("drop", pipe_drop),
                ("preprocessor", pp),
                ("model", LogisticRegression(max_iter= 10000))

])

target = "passed_exam"

df_x = df.drop(target, axis = 1)
df_y = df[target]

x_train, x_test , y_train, y_test = train_test_split(df_x, df_y, test_size= 0.5, random_state= 7)

pipe_final.fit(x_train, y_train)
y_predict = pipe_final.predict(x_test )

acc = accuracy_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
precision = precision_score(y_test, y_predict)


print("Métricas de desempeño - Regresión Logística")
print("-"*50)
print(f"Accuracy  : {acc:.3f}")
print(f"F1 Score  : {f1:.3f}")
print(f"Recall    : {recall:.3f}")
print(f"Precision : {precision:.3f}")
print("-"*50)





Métricas de desempeño - Regresión Logística
--------------------------------------------------
Accuracy  : 0.995
F1 Score  : 0.986
Recall    : 0.973
Precision : 1.000
--------------------------------------------------


In [102]:
# 1. Obtener nombres de features después del preprocesamiento
feature_names = pipe_final.named_steps["preprocessor"].get_feature_names_out()

# 2. Obtener coeficientes del modelo entrenado
coefs = pipe_final.named_steps["model"].coef_[0]

# 3. Imprimir
print("\n📌 Coeficientes de la Regresión Logística")
for name, coef in zip(feature_names, coefs):
    print(f"{name:30s} {coef:.5f}")

# Intercepto
intercept = pipe_final.named_steps["model"].intercept_[0]
print("\nIntercepto:", round(intercept, 5))


📌 Coeficientes de la Regresión Logística
categ__country_Bangladesh      -0.03318
categ__country_Brazil          0.32963
categ__country_Germany         -0.06676
categ__country_India           0.01227
categ__country_Indonesia       0.14174
categ__country_Nigeria         0.34860
categ__country_Other           -0.08869
categ__country_Pakistan        -0.11965
categ__country_UK              -0.39864
categ__country_USA             -0.15600
num__age                       0.08558
num__weeks_in_course           -0.04475
num__hours_spent_learning_per_week 0.07126
num__practice_problems_solved  -0.01633
num__projects_completed        -0.05619
num__tutorial_videos_watched   -0.00919
num__uses_kaggle               0.29860
num__participates_in_discussion_forums -0.38930
num__debugging_sessions_per_week -0.07446
num__self_reported_confidence_python -0.04129
num__final_exam_score          3.26593

Intercepto: -196.96373


#### Stepwise

In [103]:
drop_cols = ["prior_programming_experience", "student_id"]


num_var = make_column_selector(dtype_include="number")
cat_var = make_column_selector(dtype_include="object")

pipe_drop = FunctionTransformer(
                lambda x: x.drop(drop_cols, axis = 1, errors = "ignore")
                                )

pipe_cat = Pipeline(steps = [
                    ("cat_imputer", SimpleImputer(strategy= "most_frequent")),
                    ("ohe", OneHotEncoder()),
                    ]
                    )

pip_num = Pipeline(steps = [("imputer", SimpleImputer(strategy = "median"))])

pp = ColumnTransformer( transformers= [
    #("drop", "drop", drop_cols),
    ("categ", pipe_cat, cat_var),
    ("num", pip_num, num_var)],
    remainder= "passthrough"
)
#######################################################################################
sfs = SequentialFeatureSelector(
    estimator=LogisticRegression(),
    n_features_to_select=8,          # <-- define cuántas features finales quieres
    direction="backward",
    scoring="accuracy",                # o "f1", "accuracy", etc.
    #cv=StratifiedKFold(5, shuffle=True, random_state=42),
    n_jobs=-1
)


#######################################################################################

pipe_final = Pipeline(steps = [

                ("drop", pipe_drop),
                ("preprocessor", pp),
                ("backward", sfs),
                ("model", LogisticRegression(max_iter= 10000))

])

target = "passed_exam"

df_x = df.drop(target, axis = 1)
df_y = df[target]

x_train, x_test , y_train, y_test = train_test_split(df_x, df_y, test_size= 0.5, random_state= 7)

pipe_final.fit(x_train, y_train)
y_predict = pipe_final.predict(x_test )

acc = accuracy_score(y_test, y_predict)
f1 = f1_score(y_test, y_predict)
recall = recall_score(y_test, y_predict)
precision = precision_score(y_test, y_predict)


print("Métricas de desempeño - Regresión Logística")
print("-"*50)
print(f"Accuracy  : {acc:.3f}")
print(f"F1 Score  : {f1:.3f}")
print(f"Recall    : {recall:.3f}")
print(f"Precision : {precision:.3f}")
print("-"*50)


Métricas de desempeño - Regresión Logística
--------------------------------------------------
Accuracy  : 0.999
F1 Score  : 0.996
Recall    : 0.992
Precision : 1.000
--------------------------------------------------


In [110]:
# 1) Obtener el modelo ya entrenado desde el pipeline
model = pipe_final.named_steps["model"]

# 2) Obtener los nombres de las features tras el preprocesamiento
feature_names = pipe_final.named_steps["backward"].get_feature_names_out()

# 3) Obtener los coeficientes (vector 1D)
coefs = model.coef_[0]
intercept = model.intercept_[0]

# 4) Imprimir coeficientes + intercepto
print("\n📌 Coeficientes del modelo logístico:")
for name, coef in zip(feature_names, coefs):
    print(f"{name:30s} {coef:.5f}")

print(f"\nIntercepto: {intercept:.5f}")



📌 Coeficientes del modelo logístico:
x1                             0.60801
x5                             0.64138
x6                             -0.14407
x8                             -0.22768
x9                             -0.39909
x11                            -0.03134
x16                            0.38554
x20                            3.33309

Intercepto: -200.02859


In [111]:
import numpy as np
import pandas as pd

pre = pipe_final.named_steps["preprocessor"]
sfs = pipe_final.named_steps["backward"]
lr  = pipe_final.named_steps["model"]

# 1) Todos los nombres después del preprocesamiento (M)
all_names = pre.get_feature_names_out()

# 2) Máscara de selección del SFS (M -> k)
mask = sfs.get_support()

# 3) Nombres seleccionados (k)
sel_names = all_names[mask]

# 4) Coefs del modelo (k)
coefs = lr.coef_.ravel()
intercept = lr.intercept_[0]

# 5) Tabla ordenada
df_coef = pd.DataFrame({"feature": sel_names, "coef": coefs})
df_coef["abs_coef"] = df_coef["coef"].abs()
df_coef = df_coef.sort_values("abs_coef", ascending=False)

print("\n📌 Coeficientes (post-backward SFS):")
print(df_coef[["feature","coef"]].to_string(index=False))
print("\nIntercepto:", intercept)



📌 Coeficientes (post-backward SFS):
               feature      coef
 num__final_exam_score  3.333092
categ__country_Nigeria  0.641380
 categ__country_Brazil  0.608012
    categ__country_USA -0.399093
      num__uses_kaggle  0.385537
     categ__country_UK -0.227680
  categ__country_Other -0.144072
  num__weeks_in_course -0.031344

Intercepto: -200.02859027630282


In [105]:
feature_names = pipe_final.named_steps["preprocessor"].get_feature_names_out()
coefs = pipe_final.named_steps["model"].coef_[0]
intercept = pipe_final.named_steps["model"].intercept_[0]

# Construir fórmula
terms = " + ".join([f"({coef:.5f} * {name})" for name, coef in zip(feature_names, coefs)])
formula = f"logit(p) = {intercept:.5f} + {terms}"

print("\n🧮 Fórmula final del modelo:")
print(formula)

print("\n🔹 Probabilidad:")
print("p = 1 / (1 + exp(-logit(p)))")



🧮 Fórmula final del modelo:
logit(p) = -200.02859 + (0.60801 * categ__country_Bangladesh) + (0.64138 * categ__country_Brazil) + (-0.14407 * categ__country_Germany) + (-0.22768 * categ__country_India) + (-0.39909 * categ__country_Indonesia) + (-0.03134 * categ__country_Nigeria) + (0.38554 * categ__country_Other) + (3.33309 * categ__country_Pakistan)

🔹 Probabilidad:
p = 1 / (1 + exp(-logit(p)))


### revisar esto para backward

In [106]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

# Estimador base para selección
base_lr = LogisticRegression(max_iter=10000)

# Backward selection (elige k features finales)
sfs = SequentialFeatureSelector(
    estimator=base_lr,
    n_features_to_select=30,          # <-- define cuántas features finales quieres
    direction="backward",
    scoring="roc_auc",                # o "f1", "accuracy", etc.
    cv=StratifiedKFold(5, shuffle=True, random_state=42),
    n_jobs=-1
)

pipe_bw = Pipeline(steps=[
    ("drop", pipe_drop),              # tu dropper
    ("preprocessor", pp),             # tu ColumnTransformer (imputer + ohe + num)
    ("sfs", sfs),                     # selección backward (wrapper)
    ("model", LogisticRegression(max_iter=10000))
])

pipe_bw.fit(x_train, y_train)


ValueError: n_features_to_select must be < n_features.